In [ ]:
import re
import pathlib
import io

import numpy as np
import pandas as pd

In [ ]:
interaction_map = {
    'compton': 'compton-scattering',
    'pair': 'pair-production',
    'photo': 'photoelectric',
    'rayleigh': 'rayleigh-scattering',
    'triplet': 'triplet-production'
}

In [ ]:
new_data_directory = pathlib.Path('../pyegsnrc/data/xcom')
new_data_directory.mkdir(exist_ok=True, parents=True)

In [ ]:
file_options = list(pathlib.Path('xcom').glob('*.data'))
xcom_filepath = file_options[1]
xcom_filepath

In [ ]:
interaction_name = interaction_map[str(xcom_filepath).split('_')[-1].split('.')[0]]
csv_filepath = new_data_directory.joinpath(f"{interaction_name}.csv")

with open(xcom_filepath) as f:
    raw_data = f.readlines()

In [ ]:
single_integer_pattern = re.compile(r'^\s*\d+$')

row_with_single_integer = [
    bool(single_integer_pattern.match(item))
    for item in raw_data
]
single_integer_indices = np.where(row_with_single_integer)[0]

single_integer_indices

In [ ]:
lower_data_bounds_inclusive = single_integer_indices + 1
upper_data_bounds_non_inclusive = list(lower_data_bounds_inclusive[1::]-1) + [len(raw_data)]

raw_data_by_z = [
    raw_data[lower:upper]
    for lower, upper 
    in zip(lower_data_bounds_inclusive, upper_data_bounds_non_inclusive)
]

# raw_data_by_z

In [ ]:
# raw_data

In [ ]:
def get_mev_and_barn_from_raw_z_data(raw_z_data):
    data_as_string = ' '.join(raw_z_data).replace('\n', '').strip()
    data_as_list = re.split(r'\s+', data_as_string)
    data_as_array = np.array(data_as_list).astype(float)
    
    mev = np.exp(data_as_array[0::2])
    barn = np.exp(data_as_array[1::2])
    
    return mev, barn

In [ ]:
data = np.array([
    get_mev_and_barn_from_raw_z_data(item)
    for item in raw_data_by_z
])

data.shape

In [ ]:
assert np.all(data[:, 0, :] == data[0, 0, :])
mev = data[0, 0, :]

data_without_mev = data[:,1,:]
table = pd.DataFrame(data_without_mev, )

table.index = np.arange(1,101)
table.columns = mev

In [ ]:
# original file units, log(MeV) vs log(barn)

In [ ]:
# output units, MeV, and barn

In [ ]:


table.to_csv(csv_filepath)